In [35]:
import pandas as pd
import sys
import os

sys.path.append(os.path.abspath(os.path.join("../scripts")))

import load_data
from clean_data import dataCleaning

import cohere

from apis import cohere_api
apikey = cohere_api['apikey']

<IPython.core.display.Javascript object>

# Entities extraction: train set processing

In [2]:
extract_train = pd.read_json('../data/relations_dev.txt')
extract_test = pd.read_json('../data/relations_test.txt')

In [3]:
extract_train.head()

,document,tokens,relations
0,Bachelor's degree in Mechanical Engineering or...,"[{'text': 'Bachelor', 'start': 0, 'end': 8, 't...","[{'child': 4, 'head': 0, 'relationLabel': 'DEG..."
1,10+ years of software engineering work experie...,"[{'text': '10+ years', 'start': 0, 'end': 9, '...","[{'child': 4, 'head': 0, 'relationLabel': 'EXP..."
2,3+ years Swift & Objective-C and experience wi...,"[{'text': '3+ years', 'start': 0, 'end': 8, 't...","[{'child': 3, 'head': 0, 'relationLabel': 'EXP..."
3,8+ years experience in software engineering le...,"[{'text': '8+ years', 'start': 0, 'end': 8, 't...","[{'child': 5, 'head': 0, 'relationLabel': 'EXP..."
4,BS degree in Computer Science or related field...,"[{'text': 'BS', 'start': 0, 'end': 2, 'token_s...","[{'child': 3, 'head': 0, 'relationLabel': 'DEG..."


In [4]:
# tokens_processed = []
# for tokl in extract_train['tokens']:
#     new = ""
#     for d in tokl:
#         new += f"{d['entityLabel']}: {d['text']} \n"
    
#     tokens_processed.append([new])

In [5]:
tokens_processed = []
for tokl in extract_train['tokens']:
    new = {}
    for d in tokl:
        try:
            new[d['entityLabel']] = new[d['entityLabel']] + ', ' + d['text']
        except KeyError:
            new[d['entityLabel']] = f"{d['text']}"
    
    tokens_processed.append(new)

for i in range(len(tokens_processed)):
    processed = ""
    for k in tokens_processed[i].keys():
        processed += f"{k}: {tokens_processed[i][k]} \n"
    
    tokens_processed[i] = [processed]

In [6]:
extract_train['tokens_processed'] = tokens_processed

In [7]:
for i in range(len(extract_train)):
    extract_train['tokens_processed'][i] = extract_train['tokens_processed'][i][0]

In [8]:
## drop line 15 and 16 since no token availble
extract_train = extract_train.drop([15,16])

In [9]:
extract_train.index = range(len(extract_train))

In [30]:
prompt_extraction = ""
for i in [0, 1, 7, 15, 16, 18]:
    prompt_extraction += f"Document: {extract_train.iloc[i][0]}\n{extract_train.iloc[i][-1]}" + "----" + "\n"

In [12]:
extract_train.index

RangeIndex(start=0, stop=20, step=1)

## Field extraction in the test set

In [13]:
for i in range(len(extract_test)):
    extract_test['document'][i] = extract_test['document'][i].replace('\n', '')

In [14]:
extract_test.head()

,document,tokens,relations
0,"Currently holding a faculty, industry, or gove...","[{'text': 'Ph.D.', 'start': 75, 'end': 80, 'to...","[{'child': 18, 'head': 14, 'relationLabel': 'D..."
1,2+ years experience in the online advertising ...,"[{'text': '2+ years', 'start': 1, 'end': 9, 't...","[{'child': 7, 'head': 1, 'relationLabel': 'EXP..."
2,BA/BS5+ years of program or project management...,"[{'text': '5+ years', 'start': 7, 'end': 15, '...","[{'child': 11, 'head': 5, 'relationLabel': 'EX..."
3,Currently enrolled in a full-time degree progr...,"[{'text': 'Ph.D.', 'start': 801, 'end': 806, '...","[{'child': 140, 'head': 137, 'relationLabel': ..."
4,Currently enrolled in a full-time degree progr...,"[{'text': 'Ph.D.', 'start': 801, 'end': 806, '...","[{'child': 140, 'head': 137, 'relationLabel': ..."


### Extraction using the rest of the train set

In [41]:
use = extract_train.drop([0, 1, 7, 15, 16, 18])
use.index = range(len(use))
for i in range(len(use)):
    prompt_test = prompt_extraction + f"Document: {use.iloc[i][0]}\nDIPLOMA:"
    print(f"Predicition number {i+1}")
    co = cohere.Client(apikey)
    response = co.generate(
        model='xlarge',
        prompt= prompt_test,
        max_tokens=50,
        temperature=0.9,
        k=0,
        p=0.75,
        frequency_penalty=0,
        presence_penalty=0,
        stop_sequences=["----"],
        return_likelihoods='NONE')
    print('Prediction: {}'.format(response.generations[0].text))
    print('\nThis should be the true output:')
    print(use['tokens_processed'].iloc[i])  
    print('-----------------------------------------------')


Predicition number 1
Prediction:  3+ years 
DIPLOMA_MAJOR: Objective-C, Swift 
EXPERIENCE: 3+ years 
SKILLS: UI/design, UI/UX, UI/design, iOS

This should be the true output:
EXPERIENCE: 3+ years 
SKILLS: Swift & Objective-C 

-----------------------------------------------
Predicition number 2
Prediction:  BA/BS degree 
DIPLOMA_MAJOR: CS 
EXPERIENCE: 8+ years, 5+ years, 5+ years, 5+ years, 5+ years, 5+ years, 5+

This should be the true output:
EXPERIENCE: 8+ years, 5+ years 
SKILLS: software engineering, people management, managing leaders 

-----------------------------------------------
Predicition number 3
Prediction:  BS degree 
DIPLOMA_MAJOR: Computer Science 
EXPERIENCE: 7+ years, 5+ years, 2+ years, 7+ years 
SKILLS: C++, C++11

This should be the true output:
DIPLOMA: BS 
DIPLOMA_MAJOR: Computer Science 
EXPERIENCE: 7+ years, 5+ years, 2+ years 
SKILLS: C++, C++11, creating software for real-time environments, games, robotics, managing software engineers 

-------------------

### Extraction using the test set

In [40]:
tokens_processed = []
for tokl in extract_test['tokens']:
    new = {}
    for d in tokl:
        try:
            new[d['entityLabel']] = new[d['entityLabel']] + ', ' + d['text']
        except KeyError:
            new[d['entityLabel']] = f"{d['text']}"
    
    tokens_processed.append(new)

for i in range(len(tokens_processed)):
    processed = ""
    for k in tokens_processed[i].keys():
        processed += f"{k}: {tokens_processed[i][k]} \n"
    
    tokens_processed[i] = [processed]

extract_test['tokens_processed'] = tokens_processed

for i in range(len(extract_test)):
    extract_test['tokens_processed'][i] = extract_test['tokens_processed'][i][0]

In [42]:
for i in range(len(extract_test)):
    prompt_test = prompt_extraction + f"Document: {extract_test.iloc[i][0]}\nDIPLOMA:"
    print(f"Predicition number {i+1}")
    co = cohere.Client(apikey)
    response = co.generate(
        model='xlarge',
        prompt= prompt_test,
        max_tokens=50,
        temperature=0.9,
        k=0,
        p=0.75,
        frequency_penalty=0,
        presence_penalty=0,
        stop_sequences=["----"],
        return_likelihoods='NONE')
    print('Prediction: {}'.format(response.generations[0].text))
    print('\nThis should be the true output:')
    print(extract_test['tokens_processed'].iloc[i])  
    print('-----------------------------------------------')


Predicition number 1
Prediction:  Ph.D. 
DIPLOMA_MAJOR: Machine Learning, AI, Computer Science, Statistics, Applied Mathematics, Data Science 
EXPERIENCE: 1+ year(s) 
SKILLS:

This should be the true output:
DIPLOMA: Ph.D. 
DIPLOMA_MAJOR: machine learning, AI, computer science, statistics, applied mathematics, data science 
EXPERIENCE: 1+ year(s 
SKILLS: university, industry, government lab(s), AI 

-----------------------------------------------
Predicition number 2
Prediction:  BS/BA 
DIPLOMA_MAJOR: Economics, Statistics, Political Science, History, Psychology 
EXPERIENCE: 2+ years 
SKILLS: online advertising, research, statistical software, H

This should be the true output:
EXPERIENCE: 2+ years 
SKILLS: online advertising, research 
DIPLOMA: BS/BA, Masters 
DIPLOMA_MAJOR: Economics, Statistics, Political Science, History, Psychology, quantitative field 

-----------------------------------------------
Predicition number 3
Prediction:  BA/BS 
DIPLOMA_MAJOR: Computer Science 
EXPERIE

# Score prediction

In [43]:
data = load_data.load_excel_file('data/Example_data.xlsx', './', 'Sv1')

In [13]:
# data = dataCleaning(data).clean_text('Title')
# data = dataCleaning(data).clean_text('Description')
# data = dataCleaning(data).clean_text('Body')

In [44]:
## Remove unnecessary columns
data_use = data[['Domain', 'Title', "Description", 'Body', 'Analyst_Average_Score']]
data_use.head()

,Domain,Title,Description,Body,Analyst_Average_Score
0,rassegnastampa.news,Boris Johnson using a taxpayer-funded jet for ...,…often trigger a protest vote that can upset…t...,Boris Johnson using a taxpayer-funded jet for ...,0.0
1,twitter.com,"Stumbled across an interesting case, a woman f...","Stumbled across an interesting case, a woman f...","Stumbled across an interesting case, a woman f...",0.0
2,atpe-tchad.info,Marché Résines dans les peintures et revêtemen...,…COVID-19…COVID…COVID…COVID-19 et Post COVID…C...,Le rapport d’étude de marché Résines dans les ...,0.0
3,badbluetech.bitnamiapp.com,"AI drives data analytics surge, study finds",…hate raiders' linked to automated harassment ...,How to drive the funnel through content market...,0.0
4,kryptogazette.com,Triacetin Vertrieb Markt 2021: Globale Unterne...,…Abschnitten und Endanwendungen / Organisation...,Global Triacetin Vertrieb-Markt 2021 von Herst...,0.0


In [45]:
len(data_use)

10

In [47]:
train_set = data_use.head(8)
test_set = data_use.tail(2)

In [93]:
promptP = ""

for j in range(7,8):
    for i in range(len(train_set.iloc[j].index)):
        promptP = promptP + f"{train_set.iloc[j].index[i]}: {train_set.iloc[j][i]}" + "\n"

    promptP = promptP + "----" + "\n"


In [85]:
# test_set.index = range(len(test_set))
# promptnew = promptP

# for i in range(len(train_set.iloc[0].index) - 1):
#     promptnew = promptnew + f"{test_set.iloc[0].index[i]}: {test_set.iloc[0][i]}" + "\n"

# promptnew = promptnew + "Analyst_Average_Score:"

### Predictions on the test set

In [113]:
def score_classify(x):
    if x < 0:
        result = 'Error: Negative score'
    elif x >= 0 and x < 4:
        result = 'Not relevant news'
    elif x < 7:
        result = 'Relevant news'
    elif x <= 10:
        result = 'Very relevant news'
    else:
        result = 'Error: Score greater than 10'
    return result

In [137]:
for i in range(len(test_set)):
    
    promptnew = promptP

    for j in range(len(test_set.iloc[i].index) - 1):
        promptnew = promptnew + f"{test_set.iloc[i].index[j]}: {test_set.iloc[i][j]}" + "\n"
    promptnew = promptnew + "Analyst_Average_Score:"

    print(f"Predicition number {i+1}")
    co = cohere.Client(apikey)
    response = co.generate(
        model='xlarge',
        prompt= promptnew,
        max_tokens=50,
        temperature=0.9,
        k=0,
        p=0.75,
        frequency_penalty=0,
        presence_penalty=0,
        stop_sequences=["----"],
        return_likelihoods='NONE')
    real_score = test_set['Analyst_Average_Score'].iloc[i]
    predicted_score = response.generations[0].text
    print('Prediction: {}'.format(predicted_score))
    print(f'Predicted class of news: {score_classify(float(predicted_score[0:5]))}')
    print(f'\nThis should be the true output: {real_score}')
    print(f'Class of news: {score_classify(float(real_score))}')  
    print('-----------------------------------------------')

Predicition number 1
Prediction:  2.5
----
Predicted class of news: Not relevant news

This should be the true output: 0.33
Class of news: Not relevant news
-----------------------------------------------
Predicition number 2
Prediction:  1.46
----
Predicted class of news: Not relevant news

This should be the true output: 0.0
Class of news: Not relevant news
-----------------------------------------------


### Using a fine tuned model to predict the scorres

In [173]:
# from cohere.classify import Example

# co = cohere.Client(apikey)
# response = co.classify(
#     model="large",
#     inputs=[f"Domain: {data_use['Domain'].iloc[0]} \nTitle: {data_use['Title'].iloc[0]} \nDescription: {data_use['Description'].iloc[0]} \nBody: {data_use['Body'].iloc[0]}", f"Domain: {data_use['Domain'].iloc[1]} \nTitle: {data_use['Title'].iloc[1]} \nDescription: {data_use['Description'].iloc[1]} \nBody: {data_use['Body'].iloc[1]}"],
#     examples=[
#         Example(f"Domain: {data_use['Domain'].iloc[2]} \nTitle: {data_use['Title'].iloc[2]} \nDescription: {data_use['Description'].iloc[2]} \nBody: {data_use['Body'].iloc[2]}", f"{score_classify(data_use['Analyst_Average_Score'].iloc[2])}"),
#         Example(f"Domain: {data_use['Domain'].iloc[3]} \nTitle: {data_use['Title'].iloc[3]} \nDescription: {data_use['Description'].iloc[3]} \nBody: {data_use['Body'].iloc[3]}", f"{score_classify(data_use['Analyst_Average_Score'].iloc[3])}"),
#         Example(f"Domain: {data_use['Domain'].iloc[4]} \nTitle: {data_use['Title'].iloc[4]} \nDescription: {data_use['Description'].iloc[4]} \nBody: {data_use['Body'].iloc[4]}", f"{score_classify(data_use['Analyst_Average_Score'].iloc[4])}"),
#         Example(f"Domain: {data_use['Domain'].iloc[5]} \nTitle: {data_use['Title'].iloc[5]} \nDescription: {data_use['Description'].iloc[5]} \nBody: {data_use['Body'].iloc[5]}", f"{score_classify(data_use['Analyst_Average_Score'].iloc[5])}"),
#         Example(f"Domain: {data_use['Domain'].iloc[6]} \nTitle: {data_use['Title'].iloc[6]} \nDescription: {data_use['Description'].iloc[6]} \nBody: {data_use['Body'].iloc[6]}", f"{score_classify(data_use['Analyst_Average_Score'].iloc[6])}"),
#         Example(f"Domain: {data_use['Domain'].iloc[7]} \nTitle: {data_use['Title'].iloc[7]} \nDescription: {data_use['Description'].iloc[7]} \nBody: {data_use['Body'].iloc[7]}", f"{score_classify(data_use['Analyst_Average_Score'].iloc[7])}"),
#         Example(f"Domain: {data_use['Domain'].iloc[8]} \nTitle: {data_use['Title'].iloc[8]} \nDescription: {data_use['Description'].iloc[8]} \nBody: {data_use['Body'].iloc[8]}", f"{score_classify(data_use['Analyst_Average_Score'].iloc[8])}"),
#         Example(f"Domain: {data_use['Domain'].iloc[9]} \nTitle: {data_use['Title'].iloc[9]} \nDescription: {data_use['Description'].iloc[9]} \nBody: {data_use['Body'].iloc[9]}", f"{score_classify(data_use['Analyst_Average_Score'].iloc[9])}"),
#     ],
# )
# print("The confidence levels of the labels are: {}".format(response.classifications))
